# Execution Interval Method

In [ ]:
import json
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from collections import defaultdict
from libraries.utils import *
from libraries.exeint import exeInt


def get_uniquevar(raw_trace):
    ''' 
    convert the v2.2 trace into list of unique variables
    raw_trace = data from read_traces, list( (var, ts),(var, ts),(var, ts),.... )
    return:
        unique_var = list(var1,var2,...) ## list of strings
    '''
    unique_var = []
    for rt in raw_trace:
        [var, timestamp] = rt
        # print([var, timestamp])
        if var not in unique_var:
            unique_var += [var]
            # print(rt)
    return unique_var


def generate_map(unique_events):
    '''
    unique_events -> list of all the variables in the code (unique, and in order of logging)
    return:
        event_map -> takes the variable name and gives corresponding event number
        event_remap -> takes event number and gives associated variable name
    '''
    event_map = dict()
    event_remap = dict()
    for i in range(len(unique_events)):
        event_remap[i+1] = unique_events[i]
        event_map[unique_events[i]] = i+1

    return(event_map, event_remap)

def get_correct_detections(detection, ground_truth):
    '''
    detection -> list: detections from the  -> [[(var1, 0), (ts1, ts2), file_name], [], [], ...., []]
    ground_truth -> list: ground truth labels -> [[(ind1, ind2), (ts1, ts2), class], [], [], ...., []]

    return:
    y_pred -> list: [1, 1, 0, 1, 0, 0, ...., 1]
    y_true -> list: [1, 1, 1, 1, 0, 0, ...., 0]
    '''
    
    # gt_pred = defaultdict(list)      ### list of detections for each gt instance. The index of list denote its respective pred at that index, and list contains gt for that pred.
    rest_pred = [] ### list of detections that are not associated with any gt instance
    correct_pred = [] ### list of correct predictions
    y_true_ind = []
    y_pred_ind = []
    y_true = []
    y_pred = []
    # print('gt_pred:', gt_pred)
    # print(y_pred, y_true)

    if len(ground_truth) != 0:
        for gt_ind ,gt in enumerate(ground_truth):
            ind1 = gt[0]
            ind2 = gt[1]
            gt_ts1 = gt[2]
            gt_ts2 = gt[3]
            class_label = gt[4]

            if len(detection) != 0:
                tmp_pred = []
                for im, pred in enumerate(detection):
                    state1, state2 = pred[0]
                    pd_ts1, pd_ts2 = pred[1]
                    filename = pred[2]
                    # print('(gt_ts1, gt_ts2), (pd_ts1, pd_ts2)', (gt_ts1, gt_ts2), (pd_ts1, pd_ts2))

                    cond_1 = pd_ts1 > gt_ts1 and pd_ts2 < gt_ts2  ### check if the detection timestamp is within the ground truth timestamp (case 1)
                    cond_2 = pd_ts1 < gt_ts1 and pd_ts2 > gt_ts2  ### check if the gorund truth timestamp is within the detection timestamp (case 2)
                    cond_3 = pd_ts1 > gt_ts1 and pd_ts1 < gt_ts2 and pd_ts2 > gt_ts2    ### partial detection on right of the ground truths, check 5 second difference after this (case 3)
                    cond_4 = pd_ts2 < gt_ts2 and pd_ts2 > gt_ts1 and pd_ts1 < gt_ts1   ### partial detection on left of the ground truths, check 5 second difference after this (case 4)

                    
                    if cond_1 or cond_2 or cond_3 or cond_4:
                        print(gt_ind, im, cond_1, cond_2, cond_3, cond_4)
                        tmp_pred += [(im, pred)]    ### store all correct predictions that match with current gt                        

                if tmp_pred != []:
                    # print('tmp_pred', tmp_pred)
                    y_true_ind += [gt_ind]
                    ### if there are multiple correct predictions for a single gt, then choose the one that is closest to gt
                    if len(tmp_pred) > 1:
                        # print('if:', tmp_pred)
                        iou_pred = []
                        for ip, pred in tmp_pred:
                            print('ip, pred', ip, pred)
                            state1, state2 = pred[0]
                            pd_ts1, pd_ts2 = pred[1]
                            filename = pred[2]
                            ### get IOU for all detections with gt
                            iou = bbox_iou(gt_ts1, gt_ts2, pd_ts1, pd_ts2)    ### calculate the IOU between the ground truth and the detection to evaluate which is the best detection for the given gt
                            iou_pred += [iou]
                        best_pred_ind = iou_pred.index(max(iou_pred))
                        best_pred = tmp_pred[best_pred_ind]
                        print('ground_truth', gt)
                        print('best_pred:', best_pred)
                        print('y_pred_ind:', y_pred_ind)
                        # gt_pred[gt_ind] += [pred]  ### store the best detection for the given gt
                        if best_pred[0] not in y_pred_ind:
                            y_pred_ind += [best_pred[0]]
                            correct_pred += [best_pred[1]]
                            y_true += [1]
                            y_pred += [1]
                    else:
                        # print('else:', tmp_pred)
                        print('y_pred_ind:', y_pred_ind)
                        if tmp_pred[0][0] not in y_pred_ind:
                            y_pred_ind += [tmp_pred[0][0]]
                            correct_pred += [tmp_pred[0][1]]  
                            y_true += [1]
                            y_pred += [1]     
                else:
                    ### this means no detection for this gt, denots FN
                    y_true += [1]
                    y_pred += [0]
        
    ### calculate FP
    for im, pred in enumerate(detection):
        if im not in y_pred_ind:
            rest_pred += [pred]
            y_true += [0]
            y_pred += [1]

    return correct_pred, rest_pred, y_pred, y_true
   


def bbox_iou(b1_x1, b1_x2, b2_x1, b2_x2,):
    """
    Returns the IoU of two bounding boxes
    """

    # Get the coordinates of bounding boxes
    b1_y1 = b2_y1 = 0
    b1_y2 = b2_y2 = 1

    # get the corrdinates of the intersection rectangle
    inter_rect_x1 = max(b1_x1, b2_x1)
    inter_rect_y1 = max(b1_y1, b2_y1)
    inter_rect_x2 = min(b1_x2, b2_x2)
    inter_rect_y2 = min(b1_y2, b2_y2)
    # Intersection area
    inter_area = np.clip(inter_rect_x2 - inter_rect_x1 , a_min=0, a_max=None) * np.clip(
        inter_rect_y2 - inter_rect_y1 , a_min=0, a_max=None)
    # Union Area
    b1_area = (b1_x2 - b1_x1 ) * (b1_y2 - b1_y1 )
    b2_area = (b2_x2 - b2_x1 ) * (b2_y2 - b2_y1 )
    print('inter:', inter_area)
    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)

    return iou

In [ ]:
print('D1:', bbox_iou(1,100,11,20))
print('D2:', bbox_iou(10,20,11,20))

In [ ]:
np.zeros((5,1))

In [ ]:
a = {'a':1, 'b':2}
print(len(a))

## Load Data

In [ ]:
############ configuration ################
############################################

CODE = 'theft_protection'       ### application (code)
BEHAVIOUR_FAULTY = 'faulty_data'            ### normal, faulty_data
BEHAVIOUR_NORMAL = 'normal'            ### normal, faulty_data
THREAD = 'single'           ### single, multi
VER = 3                     ### format of data collection

base_dir = '../../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
normalbase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_NORMAL}'
faultybase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_FAULTY}'

print(normalbase_path)
print(faultybase_path)

In [ ]:

train_base_path = os.path.join(normalbase_path, 'train_data')
train_data_path = [os.path.join(train_base_path, x) for x in os.listdir(train_base_path)]



######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(faultybase_path)

paths_log.sort()
paths_traces.sort()
varlist_path.sort()
paths_label.sort()

# print(paths_log)
# print(paths_traces)
# print(varlist_path)
# print(paths_label)

test_data_path = paths_traces
test_label_path = paths_label

print(train_data_path)
print(test_data_path)
print(test_label_path)


In [ ]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3:
    to_number = is_consistent(varlist_path)

    if to_number != False:
        from_number = mapint2var(to_number)


In [ ]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

## Confidence Interval

__Confidence Interval:__

A confidence interval is a range around the mean that is likely to contain the true population mean. The formula for a confidence interval is mean ± margin of error mean±margin of error, where the margin of error depends on the desired confidence level and the standard error.

_Example:_

1. Choose a confidence level (e.g., 95%).
2. Calculate the standard error: standard deviation/ sqr_root(number of observations)
3. Calculate the margin of error: critical value × standard error
4. Determine the confidence interval: mean ± margin of error


In [ ]:
### initialize exeinz
ei = exeInt()

### Data Processing

In [ ]:
### get execution intervals for all variables

exe_list, filewise_exe_list = ei.get_exeint(train_data_path)

In [ ]:
### get the confidence intervals for all variables
confidence_intervals = ei.get_confinterval(exe_list)

############ calculate dynamic thresholds ############
thresholds = ei.get_dynamicthresh(exe_list)

In [ ]:
thresholds

### Visualising Thresholds

In [ ]:
#### plot exe_list to vsiualize the distribution of execution intervals
ei.viz_thresholds(exe_list, confidence_intervals, thresholds)


### Validation

In [ ]:
#### Detect anomalies in faulty traces
DIFF_VAL = 1
all_tp = []
all_fp = []
all_detections = [] ### format [file1_detection, file2_detection] -> file1_detection: [(state1, 0), (ts1, ts2), filename]  
y_pred_all = []
y_true_all = []
for test_data, test_label in zip(test_data_path, test_label_path):
    print(test_data, test_label)
    detection = ei.test_single(test_data, thresholds)   ### detection in format: [var, (ts1,ts2), file_name]
    dedup_detection, grouped_det = ei.remove_duplicates(detection, DIFF_VAL)  ### remove multiple detections for single ground truth
    detection = dedup_detection
    all_detections += [(test_data, detection, test_label)]  ### used to plot detections

    ### load ground truths
    ground_truth_raw = read_traces(test_label)
    ground_truth = ground_truth_raw['labels']
    label_trace_name = list(ground_truth.keys())[0]
    ground_truth = ground_truth[label_trace_name]
    print('ground truths:', ground_truth)
    print(len(ground_truth))

    # correct_pred, rest_pred, y_pred, y_true = get_ypred_ytrue(detection, ground_truth)  ### case1_pred, case2_pred, case34_pred, rest_pred
    correct_pred, rest_pred, y_pred, y_true = get_correct_detections(detection, ground_truth)  ### case1_pred, case2_pred, case34_pred, rest_pred

    assert( len(detection) == len(correct_pred) + len(rest_pred) )

    all_tp += [(test_data, correct_pred, test_label)]
    all_fp += [(test_data, rest_pred, test_label)]

    y_pred_all.extend(y_pred)
    y_true_all.extend(y_true)

    # break

In [ ]:
correct_pred

In [ ]:
rest_pred

In [ ]:
ground_truth[11]

In [ ]:
### Evaluation metrics

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, average_precision_score, ConfusionMatrixDisplay


# Calculate precision
precision = precision_score(y_true_all, y_pred_all)
print(f'Precision: {precision:.4f}')

# Calculate recall
recall = recall_score(y_true_all, y_pred_all)
print(f'Recall: {recall:.4f}')

# # Calculate average precision
# average_precision = average_precision_score(y_true_all, y_pred_all)
# print(f'Average Precision: {average_precision:.4f}')

# Calculate F1 score
f1 = f1_score(y_true_all, y_pred_all)
print(f"F1 Score: {f1:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true_all, y_pred_all)
print("Confusion Matrix:")
print(conf_matrix)

disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['normal', 'anomaly'])
disp.plot()

In [ ]:
all_fp

## Plot Detections

In [ ]:
### plot gt and detections
# for test_data, detections, test_label in all_detections:
for test_data, detections, test_label in all_tp:
    # print('test_data:', test_data)
    # print('detections:', detections)
    # print(test_label)

    ### prepare trace to plot
    col_data = preprocess_traces([test_data])
    all_df = get_dataframe(col_data) 
    # print(all_df[0])

    ### prepare detections to plot
    timestamps = col_data[0][1]
    print('timestamps:', timestamps)
    plot_val = []
    plot_x_ticks = []
    plot_class = []
    for det in detections:
        # print(det)
        det_ts1, det_ts2 = det[1]
        # print(det_ts1, det_ts2)

        det_ind1_pre = [ abs(t-det_ts1) for t in timestamps]
        det_ind1 = det_ind1_pre.index(min(det_ind1_pre))

        det_ind2_pre = [ abs(t-det_ts2) for t in timestamps]
        det_ind2 = det_ind2_pre.index(min(det_ind2_pre))
        # print(det_ind1, det_ind2)
        # print(timestamps[det_ind1], timestamps[det_ind2])

        plot_val += [(det_ind1, det_ind2)]
        plot_x_ticks += [(timestamps[det_ind1], timestamps[det_ind2])]
        plot_class += [0]

    plot_detections = [plot_val, plot_x_ticks, plot_class]

    ### get ground truths
    gt_plot = prepare_gt(test_label)

    ### plot
    for df in all_df:
        # print(df.columns)
        plot_single_trace(df, 
                          var_list, 
                          with_time=False, 
                          is_xticks=True, 
                          detections=plot_detections, 
                          dt_classlist=['detection'],
                          ground_truths=gt_plot,
                          gt_classlist=['gt_communication', 'gt_sensor', 'gt_bitflip'],
                          )

    # break

In [ ]:
2090

In [ ]:
1890

Observations
---
- since multiple variables are affected due to single anomaly, multiple detections are generated for each anomaly.
- This leads to multiple FP.
- To avoid this, we implement deduplication which groups the detections that are close to each other bsed on timestamp
- However, in this process along with decrease in FP, we have more False Negatives i.e. some anomalies are not detected. 

TODO:
- change deduplication stratergy, if possible